In [1]:
import os
import torch
from os.path import join
from monai.networks.nets import UNet

# Define the model architecture
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=2,
    channels=(16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2),
    num_res_units=2,
    norm="batch",
).to(device)

# Function to Convert to ONNX 
def Convert_ONNX(): 
    root_dir = "/tmp/tmpw_jhlhgy"  # Specify the root directory
    onnx_dir = join(root_dir, "models")  # Directory where you want to save the ONNX model
    os.makedirs(onnx_dir, exist_ok=True)  # Create the directory if it doesn't exist
    
    # set the model to inference mode 
    model.load_state_dict(torch.load(os.path.join(root_dir, "best_metric_model.pth")))
    model.eval() 
    
    import numpy as np

    loaded_array = np.load('data.npy')
    dummy = torch.from_numpy(loaded_array).to(device)

    # Export the model   
    onnx_file_path = join(onnx_dir, "spleen_3d_seg.onnx")
    with open(onnx_file_path, "wb") as f:
        torch.onnx.export(
            model,                                   # model being run 
            dummy,
            #torch.randn(1, 1, 160, 160, 160).to(device),# dummy input
            f,                                       # file object to save the model
            #export_params=True,                      # store the trained parameter weights inside the model file 
            #opset_version=10,                        # the ONNX version to export the model to 
            #do_constant_folding=True,                # whether to execute constant folding for optimization 
            #input_names=['modelInput'],             # the model's input names 
            #output_names=['modelOutput'],           # the model's output names 
            #dynamic_axes={'modelInput': {0: 'batch_size'}, 'modelOutput': {0: 'batch_size'}} # variable length axes 
        ) 

    print(f"Model has been converted to ONNX and saved at: {onnx_file_path}")

# Call the function to convert the model to ONNX
Convert_ONNX()


============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

Model has been converted to ONNX and saved at: /tmp/tmpw_jhlhgy/models/spleen_3d_seg.onnx


In [2]:
cp /tmp/tmpw_jhlhgy/models/spleen_3d_seg.onnx .

In [3]:
import onnxruntime as rt


sess = rt.InferenceSession("spleen_3d_seg.onnx", providers=rt.get_available_providers())
input_name = sess.get_inputs()[0].name
output_name = sess.get_outputs()[0].name
y_pred_temp = sess.run([output_name], {input_name: scaler.transform(X_test.values).astype(np.float32)}) 
y_pred_temp = np.asarray(np.squeeze(y_pred_temp[0]))
threshold = 0.995
y_pred = np.where(y_pred_temp > threshold, 1,0)

SyntaxError: unexpected EOF while parsing (719351615.py, line 10)